# <a id='toc1_'></a>[Building LLM Agents - Red Cross Helpful Information as Aid](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Building LLM Agents](#toc1_)    
  - [Introduction to LLM Agents](#toc1_1_)    
    - [What Are LLM Agents?](#toc1_1_1_)    
    - [How Do LLM Agents Work?](#toc1_1_2_)    
  - [Building a Basic LLM Agent with OpenAI](#toc1_2_)    
  - [Enhancing LLM Agents with LangChain](#toc1_3_)    
  - [Multi-Agent Systems with LangGraph](#toc1_4_)    
    - [What's a DAG?](#toc1_4_1_)    
      - [Graph](#toc1_4_1_1_)    
      - [Acyclic](#toc1_4_1_2_)    
      - [Directed](#toc1_4_1_3_)    
    - [Where else are DAGs used?](#toc1_4_2_)    
    - [Why Are DAGs Important for LLM Agents?](#toc1_4_3_)    
    - [Building our own DAG - a simple multi-agent system](#toc1_4_4_)    
  - [Types of LLM Agents](#toc1_5_)    
    - [Web Scraping Agent](#toc1_5_1_)    
    - [Web Search Agent](#toc1_5_2_)    
    - [Retrieval-Augmented Generation (RAG) Agent](#toc1_5_3_)    
    - [Tool-Using Agent](#toc1_5_4_)    
  - [Putting it all together](#toc1_6_)    
  - [Conclusion](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introduction to LLM Agents](#toc0_)

### <a id='toc1_1_1_'></a>[What Are LLM Agents?](#toc0_)
> Agents are systems that use LLMs as **reasoning engines** to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling. *(Source: [LangChain](https://python.langchain.com/docs/tutorials/agents/))*

### <a id='toc1_1_2_'></a>[How Do LLM Agents Work?](#toc0_)
Just like any other LLM. The only difference is that based on the constraints we put on LLM agents (e.g. prompt, structured output), we are able to get them to fulfill a specific role without further model finetuning. An LLM agent typically sits within a pipeline of LLM agents, like we see in [this example](https://langchain-mrkl.streamlit.app/?ref=streamlit-io-gallery-llms). 

## <a id='toc1_2_'></a>[Building a Basic LLM Agent with OpenAI](#toc0_)
Ensure you have the OpenAI Python library installed and use a `.env` file to set up your API key (`OPENAI_API_KEY`).

In [ ]:
# !pip install openai
# !pip install dotenv

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

Set up your API key and define a simple LLM agent function:


In [ ]:
message_history_1 = [{"role": "system", "content": "You are a helpful assistant"}]

In [ ]:
import openai
import os 

def llm_agent(input_, message_history, model="gpt-3.5-turbo"):
    llm = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    message_history.append({"role": "user", "content": f"{input_}"})

    # Generate a response from the chatbot model
    completion = llm.chat.completions.create(
      model=model,
      messages=message_history
    )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    return message_history

In [ ]:
llm_agent("I am hungry", message_history_1)

While this is not inherently a bad reply, it doesn't provide any specific support right away. So during this lesson, we'll try to make this better, bit by bit.

### Prompt Engineering

In [ ]:
domain_list = [
    "Shelter",
    "Health & Wellbeing",
    "Safety & Protection",
]

In [ ]:
red_cross_assistant_prompt = """You are an expert query analyzer for a Red Cross virtual assistant.
    Analyze the query to understand the user's needs, emotional state, and language.
    Pay special attention to any signs of emergency or urgent needs.

    If you detect any of these, mark as EMERGENCY:
    - Immediate danger
    - Medical emergencies 
    - Severe distress
    - Threats to basic safety

    If the query is unclear, you should return relevant domains from this list as clarification options:
    {}

    Important: You will analyze:
    1. Query clarity and type (clear/needs_clarification/emergency)
    2. Domains of need, from this list of options {} or "Other"
    3. Emotional state from language and content
    4. Language of query
    """.format(domain_list, domain_list)

In [ ]:
message_history_2 = [{"role": "system", "content": red_cross_assistant_prompt}]

In [ ]:
llm_agent("I am hungry", message_history_2)

Now the issue with this is that it doesn't provide any useful information, it just asks clarifying questions. But that's ok, we can create an LLM with...

### Structured Output

In [ ]:
from typing import List, Literal
from typing_extensions import Union
from pydantic import BaseModel # pydantic is a data validation library widely used in the industry

Domains = Literal[
    "Shelter",
    "Health & Wellbeing",
    "Safety & Protection",
]
DomainWithOther = Union[Domains, Literal["Other"]]

class QueryAnalysis(BaseModel):
    """Analysis output from Query Understanding Agent"""
    query_type: Literal["clear", "needs_clarification", "emergency"]
    domains: List[DomainWithOther]
    emotional_state: str
    language: str 

In [ ]:
def llm_agent(input_, message_history, model="gpt-4o-2024-08-06"):
    llm = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    message_history.append({"role": "user", "content": f"{input_}"})

    # Generate a response from the chatbot model
    completion = llm.beta.chat.completions.parse(
      model=model,
      messages=message_history,
      response_format=QueryAnalysis
    )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    return message_history

In [ ]:
llm_agent("I am hungry", message_history_2)

Wonderful! We've already created a useful AI agent. However, if we wanted to switch to a different provider, e.g. Anthropic, or use an open-source LLM, e.g. Llama, DeepSeek, we would need to change our code completely! 

## <a id='toc1_3_'></a>[Enhancing LLM Agents with LangChain](#toc0_)

With LangChain, we can use a lot more models without changing our code each time we change an LLM. Additionally, LangChain is able to couple multiple agents together, allowing an LLM to answer more complex queries more accurately.

In [ ]:
# !pip install langchain
# !pip install langchain_community
# !pip install -U langchain-openai
!pip install langchain-ollama


In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-3.5-turbo", model_provider="openai")
response = llm.invoke(message_history_1)
response

In [ ]:
response = llm.invoke(message_history_2)
response

In [ ]:
response = llm.with_structured_output(QueryAnalysis).invoke(message_history_2)
response

Now I was able to do the exact same thing with significantly fewer lines of code! Now let's see what other agents we can play with.

## <a id='toc1_4_'></a>[Multi-Agent Systems with LangGraph](#toc0_)
Although we can already build a multi-agent system with LangChain, we have one constraint: the sequence of actions can only be linear. LangGraph extends LangChain by enabling parallel/conditional interactions through DAGs (Directed Acyclic Graphs).

### <a id='toc1_4_1_'></a>[What's a DAG?](#toc0_)

A **Directed Acyclic Graph (DAG)** is a graph structure where nodes are connected by directed edges, and there are no cycles. This means that information flows in one direction without looping back.

#### <a id='toc1_4_1_1_'></a>[Graph](#toc0_)

A **graph** is a data structure consisting of nodes (or vertices) connected by edges. Graphs are widely used in computer science, mathematics, and AI to model relationships between entities.

![](../../../img/graph_edge_matrix.png)  
(Source: [HuggingFace](http://huggingface.co/blog/intro-graphml))

#### <a id='toc1_4_1_2_'></a>[Acyclic](#toc0_)

- **Cyclic vs. Acyclic Graphs**: A cyclic graph has cycles, whereas an acyclic graph does not.  

![](../../../img/a_cyclical_graphs.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

#### <a id='toc1_4_1_3_'></a>[Directed](#toc0_)

**Directed Graphs**: Edges have a specific direction (A → B).  

![](../../../img/directed-graph.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))

### <a id='toc1_4_2_'></a>[Where else are DAGs used?](#toc0_)

![](../../../img/airflow_dag.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

DAGs are widely used in streamlining data/ML operations that need to happen sequentially and at regular time intervals. Airflow is one of the most basic platforms where you can build this type of pipelines and monitor the success/failure of your pipelines. You can also schedule runs of pipelines, connect to cloud services, etc. Other schedulers used by data scientists are Prefect (recommended), Dask (for large-scale data processing), Kubeflow pipelines (can be overkill), DAGster. 

You'd do have to do some research before choosing any of these - Airflow is always a good starting point as it's open-source, but users complain that it's outdated compared to competitors.

### <a id='toc1_4_3_'></a>[Why Are DAGs Important for LLM Agents?](#toc0_)
- **Task Execution Flow**: DAGs define clear steps in multi-agent workflows.
- **Parallel Processing**: Independent tasks can run simultaneously.
- **Dependency Management**: Ensures tasks execute in the correct order.

LangGraph uses DAGs to structure multi-agent workflows, ensuring efficient and logical task progression.

### <a id='toc1_4_4_'></a>[Building our own DAG - a simple multi-agent system](#toc0_)

In [ ]:
# !pip install langgraph

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph.message import Message
from langchain.chat_models import ChatOpenAI

def query_understanding(state):
    response = ChatOpenAI().predict("Provide an initial analysis on AI safety.")
    return Message(response)

def agent_2(state):
    response = ChatOpenAI().predict(f"Expand on the following: {state['message']}")
    return Message(response)

workflow = StateGraph()
workflow.add_nodes(["agent_1", "agent_2"])
workflow.add_edges([("agent_1", "agent_2")])
workflow.set_entry_point("agent_1")
workflow.run()

## <a id='toc1_5_'></a>[Types of LLM Agents](#toc0_)
LLM agents can be categorized based on their purpose and capabilities. Below are some common types with code examples:

### <a id='toc1_5_1_'></a>[Web Scraping Agent](#toc0_)

In [ ]:
import requests
from bs4 import BeautifulSoup

def web_agent(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    return text[:500]  # Return a snippet

print(web_agent("https://en.wikipedia.org/wiki/Artificial_intelligence"))

### <a id='toc1_5_2_'></a>[Web Search Agent](#toc0_)

### <a id='toc1_5_3_'></a>[Retrieval-Augmented Generation (RAG) Agent](#toc0_)
Uses vector databases for enhanced responses.

In [ ]:
import faiss
import numpy as np

def rag_agent(query, stored_vectors):
    index = faiss.IndexFlatL2(stored_vectors.shape[1])
    index.add(stored_vectors)
    query_vector = np.random.random((1, stored_vectors.shape[1])).astype('float32')
    distances, indices = index.search(query_vector, k=3)
    return indices

stored_vectors = np.random.random((10, 128)).astype('float32')
print(rag_agent("What is machine learning?", stored_vectors))

### <a id='toc1_5_4_'></a>[Tool-Using Agent](#toc0_)
An agent that integrates with APIs or tools (e.g., calculator, weather API).

In [ ]:
import requests

def weather_agent(city):
    api_key = "your_api_key_here"
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"
    response = requests.get(url)
    return response.json()["current"]["temp_c"]

print(weather_agent("New York"))

## <a id='toc1_6_'></a>[Putting it all together](#toc0_)

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph.message import Message
from langchain.chat_models import ChatOpenAI

def agent_1(state):
    response = ChatOpenAI().predict("Summarize the latest AI research.")
    return Message(response)

def agent_2(state):
    response = ChatOpenAI().predict(f"Expand on this summary: {state['message']}")
    return Message(response)

workflow = StateGraph()
workflow.add_nodes(["agent_1", "agent_2"])
workflow.add_edges([("agent_1", "agent_2")])
workflow.set_entry_point("agent_1")
workflow.run()

## <a id='toc1_7_'></a>[Conclusion](#toc0_)
- **LLM agents** enable intelligent text-based automation.
- **LangChain** enhances LLMs with tools, memory, and structured pipelines.
- **LangGraph** facilitates multi-agent workflows for complex AI interactions.

With these tools, you can build powerful AI-driven applications. 🚀